In [70]:
import pandas as pd


song_data = pd.read_json("song_record.json")

In [157]:
song_data.iloc[0]

user              {'_id': {'$oid': '641b13763a63f32bb9d71460'}}
songRecord    [Z66D0F60, Z6BIWFEZ, Z6B0IC89, Z6AFUB8O, ZZF0W...
Name: 0, dtype: object

In [142]:
z = song_data["user"][0]
z.get("_id").get("$oid")

'641b13763a63f32bb9d71460'

In [133]:
c = song_data["user"].apply(pd.Series)


In [131]:
d=c["_id"].apply(pd.Series)
d

,$oid
0,641b13763a63f32bb9d71460
1,641eb85a5525ce10c474f99f


In [96]:
from collections import Counter

a = Counter(song_data["songRecord"][0])


In [102]:
a.keys()

dict_keys(['Z66D0F60', 'Z6BIWFEZ', 'Z6B0IC89', 'Z6AFUB8O', 'ZZF0WU6O', 'Z6UCWZUD', 'Z6UFF9Z6', 'ZZAAZBZE', 'ZZ9IZ7FC', 'Z6ZU6E7O', 'ZW9CFDU9', 'ZZEU80WO', 'ZW7O8W7U', 'ZWADO97U', 'Z6BU7AWO', 'Z6BWFE7F', 'Z6ABDOOI', 'Z6BU8B6O', 'ZWA7CE0B', 'ZWZD9DCB', 'ZW6CZ0OW', 'ZWZ9Z0AO', 'ZW6I7ZE7', 'Z6UBADAF', 'ZWZ9Z09E', 'ZWZ9Z0AZ', 'ZW6FODWI', 'ZW807CUD', 'ZWBUA8B8', 'ZW8WCFEA', 'Z6AFC90A', 'Z6708WAZ', 'Z69AU0IA', 'ZODCZI0O', 'ZWAD6W69', 'ZOEFE9D0', 'ZWAEIAUC', 'ZUI98BWF', 'ZZU9060B', 'ZO9D67OA', 'ZW9BBE08', 'ZW8W777E', 'Z6BIO9UF', 'Z6AFC9WD', 'Z6A9IAC7', 'ZW6D986U', 'ZWZDB788', 'ZW6W0DAW', 'ZWZ9C7EW', 'ZWZEBZCZ', 'ZW7IUW0F', 'ZWAIFBFO', 'ZZ9IZ898', 'ZW9COZ9B', 'ZW7OCDC9', 'ZU7BII9Z', 'ZW7809OU', 'Z6WDI960', 'ZW6U9UDC', 'ZW6U9UEU', 'Z69U96OW', 'ZWA80C89', 'ZWAFDUW0', 'ZUZAO6A9', 'ZW7I06CD', 'ZW7OCEOE', 'Z690CIFO', 'ZO6EIB7B', 'ZW7IUA6E', 'Z6B7AE6A', 'Z6B6CBUD', 'Z6B7AFCO', 'Z6BU6IU6', 'Z6BZWZDF', 'Z6ACA0FZ', 'Z6BOOAFC', 'Z6BO69E8', 'Z6BO6B8U', 'ZZCIOB0W'])

In [234]:
from numpy import NaN


def generate_song_record(data):
    total_df = pd.DataFrame()
    for index in range(len(data)):
        user_data = data.iloc[index]
        user_id = user_data["user"].get("_id").get("$oid")
        if (user_data["songRecord"] is not NaN):
            song_record = Counter(user_data["songRecord"])
            df = pd.DataFrame()
            df["song_id"] = song_record.keys()
            df["plays"] = song_record.values()
            df['user_id'] = user_id
            total_df = pd.concat([total_df, df])
        # user_id = user.get("_id").get("$oid")
    return total_df


In [239]:
dt = generate_song_record(song_data)
dt.set_index("user_id", inplace=True)
dt.to_csv("song_record.csv")

In [242]:
song_record_data = pd.read_csv("song_record.csv")

In [246]:
from scipy.sparse import csr_matrix

wide_song_data = song_record_data.pivot(
    index="song_id", columns="user_id", values="plays").fillna(0)

wide_song_data_sparse = csr_matrix(wide_song_data.values)


TypeError: 'csr_matrix' object is not callable

In [248]:
import numpy as np


def save_sparse_csr(filename, array):
    np.savez(filename, data=array.data, indices=array.indices,
             indptr=array.indptr, shape=array.shape)


def load_sparse_csr(filename):
    loader = np.load(filename)
    return csr_matrix((loader['data'], loader['indices'], loader['indptr']), shape=loader['shape'])

save_sparse_csr("song_data_sparse_matrix.npz",wide_song_data_sparse)


In [250]:
wide_song_data.to_csv("wide_song_data.csv")